In [ ]:
!pip install swig
!pip install wrds
!pip install PyPortfolioOpt

!pip install git+https://github.com/AI4Finance-Foundation/FinRL.git

  Cloning https://github.com/AI4Finance-Foundation/FinRL.git to /tmp/pip-req-build-r4hz1gig
  Running command git clone --filter=blob:none --quiet https://github.com/AI4Finance-Foundation/FinRL.git /tmp/pip-req-build-r4hz1gig
  Resolved https://github.com/AI4Finance-Foundation/FinRL.git to commit fb2becacbf3c48249ce960f3ef60a862f8fc6707
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Cloning https://github.com/AI4Finance-Foundation/ElegantRL.git to /tmp/pip-install-djhpb6n3/elegantrl_c0a58bb65d3048008926380cc164a774
  Running command git clone --filter=blob:none --quiet https://github.com/AI4Finance-Foundation/ElegantRL.git /tmp/pip-install-djhpb6n3/elegantrl_c0a58bb65d3048008926380cc164a774
  Resolved https://github.com/AI4Finance-Foundation/ElegantRL.git to commit b2ec84b19a82e6f18b781743d8bf37ca27134238
  Preparing metadata (setup.py) ... done
  Using cached alpaca_py-0.37.0-py3-none-any.whl.met

In [ ]:
import pandas as pd
import numpy as np
import datetime
import yfinance as yf

from finrl.meta.preprocessor.yahoodownloader import YahooDownloader
from finrl.meta.preprocessor.preprocessors import FeatureEngineer, data_split
from finrl import config_tickers
from finrl.config import INDICATORS

import itertools


Gym has been unmaintained since 2022 and does not support NumPy 2.0 amongst other critical functionality.
Please upgrade to Gymnasium, the maintained drop-in replacement of Gym, or contact the authors of your software and request that they upgrade.
See the migration guide at https://gymnasium.farama.org/introduction/migration_guide/ for additional information.


In [ ]:
TRAIN_START_DATE = '2010-01-01'
TRAIN_END_DATE = '2024-07-01'
TRADE_START_DATE = '2024-07-01'
TRADE_END_DATE = '2025-07-01'


In [ ]:
symbols = config_tickers.DOW_30_TICKER
print(symbols)

['AXP', 'AMGN', 'AAPL', 'BA', 'CAT', 'CSCO', 'CVX', 'GS', 'HD', 'HON', 'IBM', 'INTC', 'JNJ', 'KO', 'JPM', 'MCD', 'MMM', 'MRK', 'MSFT', 'NKE', 'PG', 'TRV', 'UNH', 'CRM', 'VZ', 'V', 'WBA', 'WMT', 'DIS', 'DOW']


In [ ]:
df_raw = YahooDownloader(start_date = TRAIN_START_DATE,
                     end_date = TRADE_END_DATE,
                     ticker_list = symbols).fetch_data()

YF deprecation warning: set proxy via new config function: yf.set_config(proxy=proxy)


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

Shape of DataFrame:  (114563, 8)


array([0, 1, 2, 3, 4], dtype=int32)

In [ ]:
fe = FeatureEngineer(use_technical_indicator=True,
                     tech_indicator_list=INDICATORS,
                     use_vix=True,
                     use_turbulence=True,
                     user_defined_feature=False)

processed = fe.preprocess_data(df_raw)

Successfully added technical indicators


[*********************100%***********************]  1 of 1 completed


Shape of DataFrame:  (3895, 8)
Successfully added vix
Successfully added turbulence index


In [ ]:
list_ticker = processed["tic"].unique().tolist()
list_date = list(pd.date_range(processed['date'].min(),processed['date'].max()).astype(str))
combination = list(itertools.product(list_date,list_ticker))

processed_full = pd.DataFrame(combination,columns=["date","tic"]).merge(processed,on=["date","tic"],how="left")
processed_full = processed_full[processed_full['date'].isin(processed['date'])]
processed_full = processed_full.sort_values(['date','tic'])

processed_full = processed_full.fillna(0)

In [ ]:
processed_full

,date,tic,close,high,low,open,volume,day,macd,boll_ub,boll_lb,rsi_30,cci_30,dx_30,close_30_sma,close_60_sma,vix,turbulence
0,2010-01-04,AAPL,6.431895,6.446621,6.382907,6.414463,493729600.0,0.0,0.000000,6.453184,6.421727,100.000000,66.666667,100.000000,6.431895,6.431895,20.040001,0.000000
1,2010-01-04,AMGN,39.913246,40.016969,39.111109,39.159514,5277400.0,0.0,0.000000,6.453184,6.421727,100.000000,66.666667,100.000000,39.913246,39.913246,20.040001,0.000000
2,2010-01-04,AXP,32.637970,32.781539,32.215241,32.550236,6894300.0,0.0,0.000000,6.453184,6.421727,100.000000,66.666667,100.000000,32.637970,32.637970,20.040001,0.000000
3,2010-01-04,BA,43.777557,43.941197,42.702209,43.419109,6186700.0,0.0,0.000000,6.453184,6.421727,100.000000,66.666667,100.000000,43.777557,43.777557,20.040001,0.000000
4,2010-01-04,CAT,39.403458,39.834170,38.703549,38.797770,7325600.0,0.0,0.000000,6.453184,6.421727,100.000000,66.666667,100.000000,39.403458,39.403458,20.040001,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
163961,2025-06-27,UNH,309.109985,310.450012,302.200012,302.950012,15304400.0,4.0,-9.816391,314.454535,293.444287,37.902626,66.306368,14.929805,301.604691,378.651220,16.320000,81.489152
163962,2025-06-27,V,348.609985,352.010010,345.660004,346.589996,6601900.0,4.0,-2.974180,381.138097,335.244905,49.086436,-90.737485,13.502606,359.240999,348.220052,16.320000,81.489152
163963,2025-06-27,VZ,41.636776,41.774548,41.312025,41.479322,21439600.0,4.0,-0.351723,43.932505,40.616420,47.473363,-101.931966,13.582476,42.490306,42.513984,16.320000,81.489152
163964,2025-06-27,WBA,11.450000,11.470000,11.380000,11.390000,10958600.0,4.0,0.055115,11.462499,11.190501,56.926859,136.645893,33.492015,11.290333,11.127000,16.320000,81.489152


In [ ]:
train = data_split(processed_full, TRAIN_START_DATE,TRAIN_END_DATE)
trade = data_split(processed_full, TRADE_START_DATE,TRADE_END_DATE)
print(len(train))
print(len(trade))

105734
7221


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
train_path = '/content/drive/MyDrive/SympactAI_Testing_Data/train_data.csv'
test_path = '/content/drive/MyDrive/SympactAI_Testing_Data/test_data.csv'

In [ ]:
train.to_csv(train_path)
trade.to_csv(test_path)